# Assignment 9: Supervised Machine Learning

## Learning Objectives
This lesson meets the following learning objectives:

## Instructions
Read through all of the text in this page. This assignment provides step-by-step training divided into numbered sections. The sections often contain embeded exectable code for demonstration.  Section headers with icons have special meanings:  

- <i class="fas fa-puzzle-piece"></i> The puzzle icon indicates that the section provides a practice exercise that must be completed.  Follow the instructions for the exercise and do what it asks.  Exercises must be turned in for credit.
- <i class="fa fa-cogs"></i> The cogs icon indicates that the section provides a task to perform.  Follow the instructions to complete the task.  Tasks are not turned in for credit but must be completed to continue progress.

Review the list of items in the **Expected Outcomes** section to check that you feel comfortable with the material you just learned. If you do not, then take some time to re-review that material again. If after re-review you are not comfortable, do not feel confident or do not understand the material, please ask questions on Slack to help.

Follow the instructions in the **What to turn in** section to turn in the exercises of the assginment for course credit.

## Background

For this notebook we will learn about supervised machine learning using the **scikit-learn** (**sklearn**) package. skLearn is a Python library that provides machine clearning capabilities.  It is built on Numpy, SciPy and matplotlib.

For machine learning there are three primary purposes:
- Classification: to predict the outcome "class" to which a sample belongs.
- Regression: predicting an outcome value on a continuous scale.
- Clustering: automatic grouping of similar objects into outcome classes when those classes are unknown.

You can read more at these links:

- Classification: https://en.wikipedia.org/wiki/Statistical_classification
- Prediction: https://en.wikipedia.org/wiki/Predictive_analytics
- Clustering: https://en.wikipedia.org/wiki/Cluster_analysis

For supervised machine learning, a training set of data is provided to a set of appropriate algorithms (i.e. for classification, regression or clustering). The training data set typically consists of a subset of all available data that includes both independent and dependent variables (i.e. **outcome**) measured across a variety of samples. This training data is provided to one or more algorithms which determine a **model** that can be used to classify or predict outcomes.  Typically, a set of data is set aside to test, or validate, the accuracy of the model.  

For this notebook we will once again use the Iris dataset and we will use supervised machine learning to create a model for predicting species. Therefore, the outcome variable is `species` and all others (`sepal_width`, `sepal_length`, `petal_width`, `petal_length`) are the independent variables.  Therefore, this notebook will demonstrate a "Classification" example.

## <i class="fa fa-cogs"></i> Notebook Setup
As before, we import any needed packages at the top of our notebook. Let's import Numpy, Pandas, Seaborn, matplotlib and the sklearn machine learning libraries.

In [ ]:
%matplotlib inline

# Data Management
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## 1. Data Exploration  

Import the iris dataset

In [ ]:
iris = sns.load_dataset('iris')

### 1.1 Summarize the dataset
Just as we learned in the Data Wrangling notebook, we should always summarize the data. Execute the following commands to explore this data.

In [ ]:
iris.shape

In [ ]:
iris.head(10)

In [ ]:
iris.dtypes

In [ ]:
iris.describe()

How many samples do we have for our outcome variable?

In [ ]:
iris.groupby('species').size()

### 1.2 Check for missing or duplicated data
Just like we did in the Data Wrangling notebook, we want to check for missing values and duplication

In [ ]:
iris.isna().sum()

In [ ]:
iris.duplicated().sum()

In [ ]:
iris[iris.duplicated(keep= False)]

In [ ]:
iris.nunique()

### 1.3 Examine data distributions and check for outliers
Some statistical methods make assumptions about the distribution of samples for each variable and the presence of outliers, so, we need to be aware of how our data appears and any potential outliers. Pandas dataframes have a very convenient `hist` function for printing histograms of all numeric columns. It is based on Matplotlib.

In [ ]:
iris.hist()
plt.show()

Conveniently, Pandas dataframes also have a flexible `plot` function as well, that allows us to print other types of pltos, such as a boxplot. You can learn more about the plot function by viewing the [online documentation](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html)

In [ ]:
iris.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

Let's see if there are outliers without our outcome groups. It turns out we can easily do this with a `grouby`.  The DataFrameGroupBy object, also has a `boxplot` function that makes this easy to view as well!

In [ ]:
iris.groupby(by='species').boxplot(rot=90);

It does indeed appear that we have outliers.

### 1.4 Search for Collinearity
It is important to identify collinearity prior to creating any machine learning model.  There are two types of collinearity:
- Structurual: this occurs when we create new columns in our data that are derived from other columns. For example, a log transformation of one column.
- Multicollinearity:  this can be natural in our data and occurs when one variable can be used to linearly predict another.  

Why can collinearity be bad?  For regression models, collinearity can weaken the p-values because it makes the estimated co-efficients too sensitive to changes in the model.  For an example of the affect that collinarity can have with a regression model sse the [JMP Multicollinearity](https://www.jmp.com/en_us/statistics-knowledge-portal/what-is-multiple-regression/multicollinearity.html) page.

We can empirically check for collinarity using a simple pairwise scatterplot using Seaborn.

In [ ]:
sns.pairplot(iris, hue="species")

You should consider removing columns that are severly collinear. The more collinear, the more significant the impact on the model. 

### 1.5. <i class="fas fa-puzzle-piece"></i> Practice

In the cell below notebook, perform the following.

After reviewing the data in sections 2.1, 2.2, 2.3 and 2.4 do you see any problems with this iris dataset? If so, please describe them in the practice notebook.  If not, simply indicate that there are no issues.

### 1.6. <i class="fas fa-puzzle-piece"></i> Practice

In the cell below notebook, perform the following.

After reviewing the data in sections 2.1, 2.2, 2.3 and 2.4 are there any columns that would make poor predictors of species? 

**Hint**: columns that are poor predictors are:
+ those with too many missing values
+ those with no difference in variation when grouped by the outcome class
+ variables with high levels of collinearity

## 2. Prepare the Data

### 2.1 Prepare the Data
The sklearn package expects that all indpendent variables are numerical and that independent and dependent variables are separated into different data objects. It also expects that data is in Numpy arrays (not Pandas data frames). First, let's separate the variables to only include those that are numeric. Then, we'll create a 2D Numpy array, named `X`, containing the indpendent variables.

In [ ]:
X = iris.loc[:,'sepal_length':'petal_width'].values

# Show the contents of X by displaying the first 10 rows.
X[0:10]

Next we will create a new Numpy 1D array named `Y` that will house the dependent variable (species name).

In [ ]:
Y = iris['species'].values

# Show the contents of X by displaying the first 10 elements.
Y[0:10]

Observe that we have no independent variables that are categorical. All are numeric. If we did have categorical data we must convert those to numeric values. sklearn requires that all data in `X` be numeric.  We have several options as defined in the [Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)  section of sklearn:
1. Use the sklearn `processing.OrdinalEncoder` function 
   - It converts categorical data to ordered numbers. 
   - Use this only if the classes are also ordinal.
2. Use the `processing.OneHotEncoder` function. 
   - It converts all but one class for a single variable to 0 and leaves one category of interest as 1.  
   - Use this only if you are predicting only a single class of outcome.  
3. Pivot the categorical column into multiple new binary columns. 
   - The values in the columns are 0 and 1 and indicate if the category applies to the sample row.  
   - Use this if there are multiple classes for a single variable and they are not ordinal.  Unfortunately, this is **not** tidy. But it is required to handle multiple classes.

As the iris data is all numeric we have no need for any of these options.

### 2.2 Normalize the data
Many machine learning algorithms expect that the quantitative columns are centered at 0 and scaled to unit variance.  See the [preprocessing documentation](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler) for sklearn.

According to the sklearn documentation:

> In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation... The function `scale` provides a quick and easy way to perform this operation on a single array-like dataset.

> If your data contains many outliers, scaling using the mean and variance of the data is likely to not work very well. In these cases, you can use `robust_scale`... [it uses] more robust estimates for the center and range of your data.


We can, therefore, normalize the `X` Numpy array using the [preprocessing.scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html#sklearn.preprocessing.scale) or [preprocessing.robust_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.robust_scale.html#sklearn.preprocessing.robust_scale) function of Sklearn. Choose the method most appropriate given the state of outliers in the data. As it is clear we have outliers in some of the columns of the iris data we should use the`robust_scale` method.

In [ ]:
X = preprocessing.robust_scale(X)

# Show the contents of X by displaying the first 10 rows.
# The values should be scaled between -1 to 1
X[0:10]

### 2.3  Split the data for testing and validation
For supervised machine learning, you must provide a testing dataset to create a model.  The model is then used on another dataset to validate its accuracy.  If the dataset is large enough we split the original dataset into a test and a validation set.  We can do this using the sklearn `model_selection.train_test_split` function.  The function takes as input the 2D Numpy array of independent variables and the 1D Numpy array with the oucome variable.  It also takes the following arguments:

- `test_size`: "should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split." For example, to use 80% of the data for model construction and 20% for testing (i.e. validation) this argument should be set to 0.2.  The default is 0.25.
- `random_state`: rows are randomly selected to include in the split. You can ensure the same random order by providing a seed.  This allows for reproducibility.

Lets split the data with 20% used for testing and a random seed of 10:

In [ ]:
# Split-out validation dataset
Xt, Xv, Yt, Yv = model_selection.train_test_split(X, Y, test_size=0.2, random_state=10)

# Print the shapes of each dataset
print("The sizes of the training independent and dependent datasets")
print(Xt.size)
print(Yt.size)
print("The sizes of the validation independent and dependent datasets")
print(Xv.size)
print(Yv.size)

Using the code above `Xt` and `Yt` become the "training" data.  and `Xv` and `Yv` become the "testing" data used for validation.

## 4. Perform Supervised Machine Learning


### 4.1 K-Fold Strategy

Now that we have our data separated into training and validation sets, we should establish a training strategy.  To avoid overfitting a model to the data, we should perform a K-fold cross-validation strategy.  This type of strategy will further divide our training data into <i>k</i> subsets. The model will be trained <i>k</i> times using <i>k</i>-1 subsets. The <i>k</i>th subset will be set aside for validation. But, for each of the <i>k</i> tests, a different subset will have its turn for validation.  An accuracy score is provided for each attempt.  You can evaulate the performance of a machine learning algorithm by exploring the distribution (mean and variance) of the <i>k</i> attempts (we will try this later in the notebook).

![KFCV](./media/A08-cross_validation.png)

<sup>Image from [TowardsDataScience.com Cross Validation](https://towardsdatascience.com/cross-validation-explained-evaluating-estimator-performance-e51e5430ff85) page.

To establish a K-fold cross-validation strategy we use the `model_selection.KFold` function of sklearn.  It takes two important arguments:

- `n_splits`: the number of subsets to split the data.
- `random_state`: you can ensure the same random order in the subsets by providing a seed.  This allows for reproducibility.

Lets create a K-fold strategy that splits the iris data into 10 subsets and a random seed of 10.

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=10)

### 4.2. Evaulate ML Algorithms

sklearn provides a variety of supervised machine learning aglorithms. Here, we will evaluate six of them:

1. Logistic Regression (LR)
2. Linear Discriminant Analysis (LDA)
3. K-Nearest Neighbors (KNN).
4. Classification and Regression Trees (CART).
5. Gaussian Naive Bayes (NB).
6. Support Vector Machines (SVM).

Each of these algorithms is provided by sklearn as an object. Notice, these were imported in Section 1 above: 

```python
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
```

To build a model, we must first create an object for the algorithm.  For example, to create an object for Logistic Regression we could use the following code:
```python
alg = LogisticRegression(solver='lbfgs', multi_class="auto")
```


In the following sections, we will construct an object for each algorithm and use it to create a predictive model, but first, we need a way to store the results of all ML algorithms that we'll be using. To make this easy, we will store results from each method in a dictionary. The following code prepares a dictionary containing, as keys, the names of the 6 methods we will explore.  The value is an array of 10 zeros.  This is because we will be performing a 10-fold cross-validation and we replace the zeros with results of each of the 10 attempts per method. 

In [ ]:
results = {
    'LogisticRegression' : np.zeros(10),
    'LinearDiscriminantAnalysis' : np.zeros(10),
    'KNeighborsClassifier' : np.zeros(10),
    'DecisionTreeClassifier' : np.zeros(10),
    'GaussianNB' : np.zeros(10),
    'SVC' : np.zeros(10)
}
results

After creating the algorithm object (in this example a LogisiticRegression object) we perform the model building and cross-validation in one step using the `model_selection.cross_val_score` funtion:

```python
model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, scoring="accuracy", error_score=np.nan)
```
Here we provide the algorithm object, the training data, validation data, k-fold object, the `scoring` accuracy argument and an `error_score` argument.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) for details about other arguments.  We can use the same `cross_val_score` function for any of the 6 algorithms.

In the following sections, six supervised machine learning methods will be briefly introduced.  For brevity we will not explore these approaches too deeply. A brief summary is provided followed by links for additional details.

#### 4.2.1 Logistic Regression

**Brief Summary** 

In statistics, the logistic model (or logit model) uses a logistic function to model a binary outcome (i.e. values of 0 and 1) and a set of one or more independent variables: [X<sub>1</sub>, X<sub>2</sub>, X<sub>3</sub>, ... X<sub>n</sub>]:

\begin{equation*}
logit(p) = b_0 + b_1X_1 + b_2X_2 + b_2X_2 ... b_nX_n
\end{equation*}

Its goal is to determine the set of coefficients, [b<sub>1</sub>, b<sub>2</sub>, b<sub>3</sub>, ... b<sub>n</sub>], that best fit the relationship between the dependent and independent variables.   Alternatively, it is the natural log of the odds ratio:

\begin{equation*}
odds\ ratio = \frac{p}{1-p} = \frac{probability\ of\ presence}{probability\ of\ absense}
\end{equation*}

\begin{equation*}
logit(p) = ln(\frac{p}{1-p})
\end{equation*}


The following image from Wikipedia shows an example where the dependent variable of *"passing an exam"* with a single independent variable of *"hours studied"*. 

<img src="./media/A08-Exam_pass_logistic_curve.jpeg" style="height:300px">

Samples can be classified as predicting a passing score or a non-passing score by where they fall (above or below) on the line.  As a machine learning approach, we can provide a set of training data to create the model, then use future data to predict an outcome.

**When to use:**
+ Input data is quantitative.
+ No colliniearity.
+ No outliers.
+ Relationship is expected to be linear

**Additional Resources**
+ [skLearn Logisitic Regression Function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
+ [Wikipedia LR](https://en.wikipedia.org/wiki/Logistic_regression)
+ [TowardsDataScience.com LR](https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8)

**Practice**

To perform Logisitc Regression with sklearn, we must first create a LogisiticRegression object. There are two important arguments to provide:

- `solver`: There are several different solvers: ‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’.  These are meant for different types of data:
   + For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones.
   + For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.
   + ‘newton-cg’, ‘lbfgs’ and ‘sag’ only handle L2 penalty, whereas ‘liblinear’ and ‘saga’ handle L1 penalty.
- `multi_class`: set this argument when there are more than two classes in the outcome variable. Setting it to `auto` allows the algorithm to select the best approach for working with the data.  

Because the iris species is multinomial (multiple categories), we will set the `multi_class` argument to `auto` and the `solver` to `libfgs`

In [ ]:
# Create the LogisticRegression object prepared for a multinomial outcome validation set.
alg = LogisticRegression(solver='lbfgs', multi_class="auto")

# Execute the cross-validation strategy
results['LogisticRegression'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                scoring="accuracy", error_score=np.nan)

# Take a look at the scores for each of the 10-fold runs.
results['LogisticRegression']

#### 4.2.2 Linear Discriminant Analysis (LDA)

**Brief Summary** 

Linear Discriminant Analysis (LDA) is a classification method that employs a dimensionality reduction technique.  

From the [Wikipedia LDA](https://en.wikipedia.org/wiki/Linear_discriminant_analysis) page:

> Linear discriminant analysis (LDA),... [is] a method used in statistics, pattern recognition and machine learning to find a linear combination of features that characterizes or separates two or more classes of objects or events... LDA is closely related to analysis of variance (ANOVA) and regression analysis, which also attempt to express one dependent variable as a linear combination of other features or measurements. However, ANOVA uses categorical independent variables and a continuous dependent variable, whereas discriminant analysis has continuous independent variables and a categorical dependent variable (i.e. the class label)

Assumptions:
+ Outcome classes are normally distributed.
+ Variance between outcome classes is equal.

LDA is similar to Principal Component Analysis (PCA), but….
+ PCA is unsupervised, LDA is supervised
+ PCA tries to maximize the variance.
+ LDA tries to maximize the separation between data classes.

<img src="./media/A08-lda_1.png" style="height:250px">
<sup><i>Image from <a href="http://sebastianraschka.com/Articles/2014_python_lda.html">SebastianRaschka Liniear Discriminant Analysis</a> page</i></sup>

**When to use:**
+ The dependent variable is categorical (qualitative) with "classes" or categories as outcomes 
+ The independnet variables are quantitative.

**Additional Resources**
+ [skLearn LDA](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)
+ [Wikipedia LDA](https://en.wikipedia.org/wiki/Linear_discriminant_analysis)
+ [TowardsDataSciecne LDA](https://towardsdatascience.com/classification-part-2-linear-discriminant-analysis-ea60c45b9ee5)

**Practice**

Similar to Logistic Regression, we must first create a LinearDiscriminantAnalysis object, then call `cross_val_score`.  Here we'll just use the default settings.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) for a more thorough description of arguments and flexibility of the algorithm.

In [ ]:
# Create the LinearDiscriminantAnalysis object with defaults.
alg = LinearDiscriminantAnalysis()

# Execute the cross-validation strategy
results['LinearDiscriminantAnalysis'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                        scoring="accuracy", error_score=np.nan)

# Take a look at the scores for each of the 10-fold runs.
results['LinearDiscriminantAnalysis']

#### 4.2.3 K-Nearest Neighbors (KNN)

**Brief Summary**

From the [Wikipedia KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) page:

> In pattern recognition, the k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification and regression... 
>
> <img src="./media/A08-KnnClassification.svg" style="height: 200px">
>
> [Above is an] example of k-NN classification. The test sample (green circle) should be classified either to the first class of blue squares or to the second class of red triangles. If k = 3 (solid line circle) it is assigned to the second class because there are 2 triangles and only 1 square inside the inner circle. If k = 5 (dashed line circle) it is assigned to the first class (3 squares vs. 2 triangles inside the outer circle).

For the KNN approach to be useful, the outcome classes must be distinguishable. As an exapmle of distinguishable classes, consider the Iris dataset petal width vs petal length:

In [ ]:
sns.relplot(x="petal_width", y="petal_length", hue="species", data=iris);

**When to Use**
+ Multiple outcomes (not just binary)
+ There are no assumptions for the outcome distribution
+ The outcomes classes are already distinguishable in the problem space.

**Additional Resources**
+ [skLearn KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
+ [Wikipedia KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)
+ [TowardsDataScience.com KNN](https://towardsdatascience.com/knn-k-nearest-neighbors-1-a4707b24bd1d)

**Practice**

Similar to other algorithms, we must first create a KNeighborsClassifier object, then call `cross_val_score`.  Here we'll just use the default settings.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) for a more thorough description of arguments and flexibility of the algorithm.

In [ ]:
# Create the KNeighborsClassifier object with defaults.
alg = KNeighborsClassifier()

# Execute the cross-validation strategy
results['KNeighborsClassifier'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                  scoring="accuracy", error_score=np.nan)
# Take a look at the scores for each of the 10-fold runs.
results['KNeighborsClassifier']

#### 4.2.4  Classification and Regression Trees (Decision Trees)

**Brief Summary** 

From the [Wikipedia Decision Tree Learning](https://en.wikipedia.org/wiki/Decision_tree_learning) page:

> In computer science, Decision tree learning uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). It is one of the predictive modeling approaches used in statistics, data mining and machine learning. Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.
>
>![Decsion Trees](./media/A08-CART_tree_titanic_survivors.png)
>
> [The above] tree [shows] survival of passengers on the Titanic. The figures under the leaves show the probability of survival and the percentage of observations in the leaf.

In short, trees are learned by splitting the source data based on an attribute value and two types of trees are possible based on the outcome data:

Classification Trees
- When outcome is categorical

Regression Trees
- When outcome is continuous

**When to Use**
- For both numerical and categorical outcome data
- Distribution of outcome is unknown or doesn’t meet other model assumptions.
- Multiple outcomes.

**Additional Resources**
- [sklearn Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [sklearn CART](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [Wikipediea Decision Trees](https://en.wikipedia.org/wiki/Decision_tree)
- [TowardsDataScience.com CART](https://towardsdatascience.com/decision-trees-d07e0f420175)


**Practice**

Similar to other algorithms, we must first create a DecisionTreeClassifier object, then call `cross_val_score`.  Here we'll just use the default settings.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for a more thorough description of arguments and flexibility of the algorithm.

In [ ]:
# Create the DecisionTreeClassifier object with defaults.
alg = DecisionTreeClassifier()

# Execute the cross-validation strategy
results['DecisionTreeClassifier'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                  scoring="accuracy", error_score=np.nan)
# Take a look at the scores for each of the 10-fold runs.
results['DecisionTreeClassifier']

#### 4.2.5 Gaussian Naive Bayes (NB)

**Brief Summary**

From the [Wikipedia Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) page:

> In machine learning, naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naive) independence assumptions between the features.

Naive Bayes methods rely on Bayes theorem, and follows axioms of the conditional probability:

\begin{equation*}
P(A | B) = \frac{P(B|A)P(A)}{P(B)}
\end{equation*}

.
Where:
 - A = outcome class
 - B = input data vector for a sample.
 
There are many NB algorithms and they differ in assumptions about the distribution of P(A|B). For the Guassian approach the distributions are expected to be Guassian and the input data is quantitative.

**When to Use**

- High independence between input data (no multicollinearity)
- Multiple outcome classes.
- Independent variables are quantitative.

**Additional Resources**
- [skLearn Guassian NB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
- [Wikipedia NB](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)
- [TowardsDataScience NB](https://towardsdatascience.com/all-about-naive-bayes-8e13cef044cf)

**Practice**

Similar to other algorithms, we must first create a GaussianNB object, then call `cross_val_score`.  Here we'll just use the default settings.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) for a more thorough description of arguments and flexibility of the algorithm.

In [ ]:
# Create the GaussianNB object with defaults.
alg = GaussianNB()

# Execute the cross-validation strategy
results['GaussianNB'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                  scoring="accuracy", error_score=np.nan)
# Take a look at the scores for each of the 10-fold runs.
results['GaussianNB']

#### 4.2.6 Support Vector Machines (SVM)
**Brief Summary**

A support vector machine attempts to find an optimal separation between outcomes by separating them in multi-dimensional space (&reals;<sup>n</sup>). Consider the following figure:

<img src="./media/A08-Hyperplane.png" style="height:250px">
<sup><i>Image from <a href="https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47">TowardsDataScience.com SVM</a> page</i></sup>

Here the line in the first 2-dimensional plot separates the two classes and in the 3-dimensional plot, the plane separates them.  For larger dimensions, SVMs find the optimal **hyperplane**.  The optimal "hyperplane" (line or plane) is selected as the one which maximizes the **margin** between the outcome classes.  Consier the following figure:

<img src="./media/A08-Svm_separating_hyperplanes.svg" style="height:250px">
<sup><i>Image from <a href="https://en.wikipedia.org/wiki/Support-vector_machine">Wikipedia SVM</a> page</i></sup>

In the example figure above:
- The H1 line does not separate the classes. 
- The H2 line does, but only with a small margin (distance between samples in the classes) 
- The H3 line separates them with the maximal margin.

There are multiple types of SVMs.  The SVM that performs classification is known as a Support Vector Classification (SVC) algorithm.

**When to use**
- Multiple outcome classes.
- Distribution of outcome is unknown or doesn’t meet other model assumptions.
- High dimensional data
- There are more samples than features (otherwise overfitting may occur)

**Additional Resources**
- [skLearn SVM](https://scikit-learn.org/stable/modules/svm.html)
- [skLearn SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
- [Wikipedia SVM](https://en.wikipedia.org/wiki/Support-vector_machine)
- [TowardsDataScience.com SVM](https://towardsdatascience.com/support-vector-machines-a-brief-overview-37e018ae310f)


**Practice**

Similar to other algorithms, we must first create a SVC object, then call `cross_val_score`.  Here we'll just use the default settings.  See the [online documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) for a more thorough description of arguments and flexibility of the algorithm.

In [ ]:
# Create the SVC object with defaults.
alg = SVC(gamma='auto')

# Execute the cross-validation strategy
results['SVC'] = model_selection.cross_val_score(alg, Xt, Yt, cv=kfold, 
                                                                  scoring="accuracy", error_score=np.nan)
# Take a look at the scores for each of the 10-fold runs.
results['SVC']

### 4.3 Compare Scores
Now that we have performed training and validation of six different methods we can compare the results to see which performed best.  We can do so by converting the results dictionary into a dataframe and using the `plot` function that comes with Panda's data frames to create a box plot of each test.

In [ ]:
pd.DataFrame(results).plot(kind="box", rot=45);

### 4.4 Test the model (make predictions)

From the boxplot in the previous section, it seems as if Linear Discriminant Analysis performed best!  We will use this algorithm and the trained model to compare predictions made from the 20% of data we set aside for testing match the actual outcomes.  To do this we will use two new functions that are members of any of the algorithm objects:

- `fit`: This function employs the algorithm to create the final predictive model. All the training data is provided to this function. 
- `predict`:  This function uses the trained model to make predictions using the testing data.

In [ ]:
# Create the LinearDiscriminantAnalysis object with defaults.
alg = LinearDiscriminantAnalysis()

# Create a new model using all of the training data.
alg.fit(Xt, Yt)

# Using the testing data, predict the iris species.
predictions = alg.predict(Xv)

# Let's see the predictions
predictions

Finally, we can evaulate how accurate the model has been by comparing the predictions with the actual species for the test data.  The `accuracy_score` function provides this:

In [ ]:
accuracy_score(Yv, predictions)

One way to explore the performance of the algorithm is by way of a **Confusion Matrix** or error matrix.  A confusion matrix is an <i>n</i> x <i>n</i> matrix where n is the number of outcome classes. In the case of the Iris data, <i>i</i> = 3.  To view the confusion matrix use the sklearn `confusion_matrix` function.  We pass in the testing outcome set, the predictions and the order that we want the classes to appear (using the `labels` argument):

In [ ]:
labels = ['versicolor', 'virginica', 'setosa']
cm = confusion_matrix(Yv, predictions, labels=labels)
print(cm)

The elements of the confusion matrix have the following meaning:
- rows of the confusion matrix represent the predicted classes 
- columns represent the actual classes.  
- elements on the diagnoal of the matrix represent the true positives
- errors are present when counts above zero are outside of the diagnoal.

***Note***: Because the result is a Numpy array, there are no row and column labels. However, because we provided the `labels` argument we know the order of the classes in the matrix.

We can use the Seaborn `heatmap` to visualize the matrix:

In [ ]:
sns.heatmap(cm, annot=True, xticklabels=labels, yticklabels=labels);

Finally, the `classification_report` function indicates how well the model was at predicting each class.

In [ ]:
cr = classification_report(Yv, predictions)
print(cr)

Where:
- `precision`: The precision is the ratio `tp / (tp + fp)` where `tp` is the number of true positives and `fp` the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
- `recall`: The recall is the ratio `tp / (tp + fn)` where `tp` is the number of true positives and `fn` the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
- `f1-score`: The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.
- `support`: the number of actual samples with the given outcome.

For comparison, let's examine a less performant model to see how the confusion matrix and reports indicate error.  Let's use the K-neighbors classifier

In [ ]:
# Create the LinearDiscriminantAnalysis object with defaults.
alg = KNeighborsClassifier()

# Create a new model using all of the training data.
alg.fit(Xt, Yt)

# Using the testing data, predict the iris species.
predictions = alg.predict(Xv)

# Let's see the predictions
predictions

In [ ]:
accuracy_score(Yv, predictions)

In [ ]:
labels = ['versicolor', 'virginica', 'setosa']
cm = confusion_matrix(Yv, predictions, labels=labels)
sns.heatmap(cm, annot=True, xticklabels=labels, yticklabels=labels);

In [ ]:
cr = classification_report(Yv, predictions)
print(cr)

Here we have one prediction that is false where `virginica` was predicted to be `versicolor`

### 4.5. <i class="fas fa-puzzle-piece"></i> Practice

In the cell below notebook, perform the following.

Now that you have learned how to perform supervised machine learning using a variety of algorithms, lets practice using a new algorithm we haven't looked at yet: the Random Forest Classifier.  The random forest classifier builds multiple decision trees and merges them together.  Review the sklearn [online documentation for the RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).  For this task:

1. Perform a 10-fold cross-validation strategy to see how well the random forest classifier performs with the iris data
2. Use a boxplot to show the distribution of accuracy
3. Use the `fit` and `predict` functions to see how well it performs with the testing data.
4. Plot the confusion matrix
5. Print the classification report.


## Expected Outcomes
At this point, you should feel comfortable with the following:
 - A basic understanding of popular supervised machine learning methods.
 - Preparing and normalizing data for supervised machine learning.
 - Splitting the data for testing and validation
 - k-fold strategy to training data.
 - Comparing scores
 - Testing the model.

## What to Turn in?
Be sure to **commit** and **push** your changes to this notebook.  All practice exercises should be completed.  Once completed, send a **Slack message** to the instructor indicating you have completed this assignment. The instructor will verify all work is completed. 